In [1]:
import pickle
import networkx as nx
import matplotlib.pyplot as plt
import torch
import dgl
import numpy as np
import torch.nn.functional as F
import torch as th
import torch.nn as nn
from roadModel import roadModel
import envPressureSubsEdges
from scipy.special import softmax

from tensorboardX import SummaryWriter


macrostep = 10
DEVICE = "cuda:1"

In [2]:
with open("stoppedEdges.pkl", 'rb') as f:
    stoppedEdges = pickle.load(f)
    
with open("amatrix_edges.pkl", 'rb') as f:
    A = pickle.load(f)
    

In [3]:
indices = {c:i for i,c in enumerate(list(A.columns))}
invertedIndices = {i:c for i,c in enumerate(list(A.columns))}
g = dgl.DGLGraph(np.eye(A.values.shape[0]) + A.values)
N = g.number_of_nodes()
embedding_n = 32

In [4]:
environ = envPressureSubsEdges.CornicheEnv(macrostep=macrostep,predefined=False,scale=8,GUI=False)
obs = environ.reset()

Could not connect to TraCI server at localhost:51313 [Errno 111] Connection refused
 Retrying in 1 seconds


## Set nodes and edges attributes

In [5]:
g.edata['prob'] = torch.ones((g.number_of_edges(),1)).cuda().to(DEVICE)
g.ndata['embedding'] = torch.ones((g.number_of_nodes(),embedding_n)).cuda().to(DEVICE)

In [6]:
class PredictLeavingModule(nn.Module):
    def __init__(self, in_feats, embedding_n):
        super(PredictLeavingModule, self).__init__()
        self.embed = nn.Embedding(in_feats, embedding_n)
        self.L2 = nn.Linear(embedding_n+1, 1)
        self.A2 = F.relu
        

    def forward(self, node):
        embedding = self.embed(node.data['features'])        
        #stack cars with embedding
        l2input = th.cat((embedding,node.data['cars']),dim=1)

        leaving_cars = self.L2(l2input)
        leaving_cars = self.A2(leaving_cars)
        
        leaving_cars = leaving_cars * node.data['free']
        leaving_cars = th.min(leaving_cars,node.data['cars'])
                
        return {'leaving_cars' : leaving_cars, 'embedding': embedding}

In [8]:
class PredictEdgeProbModule(nn.Module):
    def __init__(self, embedding_n):
        super(PredictEdgeProbModule, self).__init__()
        self.L2 = nn.Linear(embedding_n*2, 1)
        self.A2 = F.relu

    def forward(self, edge):
        h = torch.cat([edge.src['embedding'], edge.dst['embedding']], dim=1)
        h = self.L2(h)
        logit = self.A2(h)
        
        return {'logit' : logit}

In [9]:
def flow_message_func(edges):
    return {'c' : edges.src['leaving_cars']*edges.data['prob'], 'e' : edges.src['embedding']}

In [10]:
def flow_reduce_func(nodes):
    incoming = torch.sum(nodes.mailbox['c'], dim=1)
    cars =  incoming + nodes.data["cars"]
    
    a = th.softmax(th.cat((nodes.mailbox['c'], nodes.data["cars"].unsqueeze(1)),dim=1),dim=1)
    embeddings = th.cat((nodes.mailbox['e'],nodes.data['embedding'].unsqueeze(1)),dim=1)

    
    
    embedding = torch.sum(embeddings*a,dim=1)
    
    
    return {'cars' : cars, 'embedding': embedding}

In [11]:
def flow_apply_func(nodes):
    cars = nodes.data["cars"] - nodes.data["leaving_cars"]
    return {'cars':cars}

In [12]:
def softmax_feat(edges): return {'prob': th.softmax(edges.data['logit'], dim=1)}

In [13]:
class GCN(nn.Module):
    def __init__(self, nodes_feats_size, embedding_n):
        super(GCN, self).__init__()
        self.predict_parking = PredictParkingModule(embedding_n)
        self.predict_leaving = PredictLeavingModule(nodes_feats_size, embedding_n)
        self.predict_edge_prob = PredictEdgeProbModule(embedding_n)

    def forward(self, g, nfeatures):
        g.ndata['features'] = nfeatures
        g.ndata["leaving_cars"] = th.zeros((N,1)).cuda().to(DEVICE)
        
        g.apply_nodes(func=self.predict_leaving)
        g.apply_edges(func=self.predict_edge_prob)
        g.group_apply_edges(func=softmax_feat, group_by='src')
        
        g.pull(v=g.nodes(),message_func=flow_message_func, reduce_func=flow_reduce_func)
        g.apply_nodes(v=g.nodes(),func=flow_apply_func)
        
        g.apply_nodes(v=g.nodes(),func=self.predict_parking)
        

        return g.ndata.pop('cars'),g.ndata.pop('embedding')

In [14]:
class Net(nn.Module):
    def __init__(self, nfeatures_size, embedding_n):
        super(Net, self).__init__()
        self.gcn = GCN(nfeatures_size, embedding_n)
    
    def forward(self, g, nfeatures, cars, freeEdges):
        g.ndata['cars'] = cars
        g.ndata['free'] = freeEdges
        x = self.gcn(g, nfeatures)
        return x

In [15]:
net = Net(g.number_of_nodes(),embedding_n)
net.cuda().to(DEVICE)
print(net)

Net(
  (gcn): GCN(
    (predict_parking): PredictParkingModule(
      (L2): Linear(in_features=33, out_features=1, bias=True)
    )
    (predict_leaving): PredictLeavingModule(
      (embed): Embedding(1868, 32)
      (L2): Linear(in_features=33, out_features=1, bias=True)
    )
    (predict_edge_prob): PredictEdgeProbModule(
      (L2): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)


In [16]:
import time
import numpy as np

nfeatures = th.eye(g.number_of_nodes())
nids = th.LongTensor([x for x in range(g.number_of_nodes())])

MSE = nn.MSELoss()

optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
dur = []

In [17]:
TL = len(environ.action_space)

In [18]:
def send_graph_to_device(g, DEVICE):
    # nodes
    labels = g.node_attr_schemes()
    for l in labels.keys():
        g.ndata[l] = g.ndata.pop(l).cuda().to(DEVICE, non_blocking=True)
    
    # edges
    labels = g.edge_attr_schemes()
    for l in labels.keys():
        g.edata[l] = g.edata.pop(l).cuda().to(DEVICE, non_blocking=True)
    return g

In [ ]:
#g = send_graph_to_device(g,DEVICE)
epochs = 1

for run in range(5):
    obs = environ.reset(restart=(run>0))
    future_cars,phaseobs = obs
    cars = th.FloatTensor(future_cars).unsqueeze(1)
    writer = SummaryWriter(comment="Corniche_GATFlow_OneOmbedding_4steps_%depochs_%d s_Episode%d_002_scale8" % (epochs,macrostep,run))
    
    for i in range(int(24*60*60/macrostep)-1):
        #action = [environ.action_space[tl].sample() for tl in range(TL)]
        action = [ np.random.choice(list(range(environ.action_space[i].n)),p=softmax(phaseobs[i])) for i in range(len(phaseobs))]
        freeEdges = 1 - np.sum([stoppedEdges[tl][action[tl]].values.squeeze() for tl in range(TL)], axis=0)
        freeEdges = th.FloatTensor(freeEdges).unsqueeze(1)
        obs,_,_,measures = environ.step(action)
        future_cars,phaseobs = obs
        for epoch in range(epochs):
            predicted,embedding = net(g, nids.cuda().to(DEVICE), cars.cuda().to(DEVICE), freeEdges.cuda().to(DEVICE))
            predicted,embedding = net(g, nids.cuda().to(DEVICE), predicted, freeEdges.cuda().to(DEVICE))
            predicted,embedding = net(g, nids.cuda().to(DEVICE), predicted, freeEdges.cuda().to(DEVICE))
            predicted,embedding = net(g, nids.cuda().to(DEVICE), predicted, freeEdges.cuda().to(DEVICE))
            loss = MSE(predicted, th.FloatTensor(future_cars).unsqueeze(1).cuda().to(DEVICE))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            wrong_cars = np.sum(np.abs(predicted.squeeze().squeeze().detach().cpu().numpy() - future_cars))
            cars_total = np.sum(future_cars)
            wrong_cars_adjusted = wrong_cars - (measures['arrived'] + measures['departed'])


            
            if epoch == 0:
                writer.add_scalar("loss", loss.item(), i)
                writer.add_scalar("wrong_cars", wrong_cars, i)
                writer.add_scalar("wrong_cars_adjusted", wrong_cars_adjusted, i)
                writer.add_scalar("wrong_percentage", min(wrong_cars/cars_total,1.0), i)
                writer.add_scalar("total_cars", cars_total, i)
                writer.add_scalar("departed", measures['departed'], i)
                writer.add_scalar("arrived", measures['arrived'], i)

            if epoch == 0 and i%6==0:
                print("Run {:d} | Step {:d} | Epoch {:05d} | Loss {:.4f}".format(run,i,
                            epoch, loss.item()))

        cars = th.FloatTensor(future_cars).unsqueeze(1)
                    

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/dgl/base.py:18: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars


Run 0 | Step 0 | Epoch 00000 | Loss 0.6685
Run 0 | Step 6 | Epoch 00000 | Loss 0.5276
Run 0 | Step 12 | Epoch 00000 | Loss 0.4344
Run 0 | Step 18 | Epoch 00000 | Loss 0.3458
Run 0 | Step 24 | Epoch 00000 | Loss 0.2881
Run 0 | Step 30 | Epoch 00000 | Loss 0.2863
Run 0 | Step 36 | Epoch 00000 | Loss 0.4074
Run 0 | Step 42 | Epoch 00000 | Loss 0.2632
Run 0 | Step 48 | Epoch 00000 | Loss 0.3543
Run 0 | Step 54 | Epoch 00000 | Loss 0.4656
Run 0 | Step 60 | Epoch 00000 | Loss 0.3076
Run 0 | Step 66 | Epoch 00000 | Loss 0.3113
Run 0 | Step 72 | Epoch 00000 | Loss 0.3411
Run 0 | Step 78 | Epoch 00000 | Loss 0.3494
Run 0 | Step 84 | Epoch 00000 | Loss 0.4680
Run 0 | Step 90 | Epoch 00000 | Loss 0.3491
Run 0 | Step 96 | Epoch 00000 | Loss 0.5108
Run 0 | Step 102 | Epoch 00000 | Loss 0.3735
Run 0 | Step 108 | Epoch 00000 | Loss 0.4207
Run 0 | Step 114 | Epoch 00000 | Loss 0.3489
Run 0 | Step 120 | Epoch 00000 | Loss 0.3936
Run 0 | Step 126 | Epoch 00000 | Loss 0.3779
Run 0 | Step 132 | Epoch 0000

Run 0 | Step 1098 | Epoch 00000 | Loss 0.1387
Run 0 | Step 1104 | Epoch 00000 | Loss 0.1020
Run 0 | Step 1110 | Epoch 00000 | Loss 0.1091
Run 0 | Step 1116 | Epoch 00000 | Loss 0.0829
Run 0 | Step 1122 | Epoch 00000 | Loss 0.0663
Run 0 | Step 1128 | Epoch 00000 | Loss 0.1126
Run 0 | Step 1134 | Epoch 00000 | Loss 0.1193
Run 0 | Step 1140 | Epoch 00000 | Loss 0.0655
Run 0 | Step 1146 | Epoch 00000 | Loss 0.0769
Run 0 | Step 1152 | Epoch 00000 | Loss 0.0838
Run 0 | Step 1158 | Epoch 00000 | Loss 0.0741
Run 0 | Step 1164 | Epoch 00000 | Loss 0.0987
Run 0 | Step 1170 | Epoch 00000 | Loss 0.1018
Run 0 | Step 1176 | Epoch 00000 | Loss 0.0832
Run 0 | Step 1182 | Epoch 00000 | Loss 0.0837
Run 0 | Step 1188 | Epoch 00000 | Loss 0.1120
Run 0 | Step 1194 | Epoch 00000 | Loss 0.0637
Run 0 | Step 1200 | Epoch 00000 | Loss 0.0720
Run 0 | Step 1206 | Epoch 00000 | Loss 0.0909
Run 0 | Step 1212 | Epoch 00000 | Loss 0.0713
Run 0 | Step 1218 | Epoch 00000 | Loss 0.0783
Run 0 | Step 1224 | Epoch 00000 | 

Run 0 | Step 2172 | Epoch 00000 | Loss 0.1476
Run 0 | Step 2178 | Epoch 00000 | Loss 0.0977
Run 0 | Step 2184 | Epoch 00000 | Loss 0.1032
Run 0 | Step 2190 | Epoch 00000 | Loss 0.1083
Run 0 | Step 2196 | Epoch 00000 | Loss 0.1221
Run 0 | Step 2202 | Epoch 00000 | Loss 0.1671
Run 0 | Step 2208 | Epoch 00000 | Loss 0.0997
Run 0 | Step 2214 | Epoch 00000 | Loss 0.1150
Run 0 | Step 2220 | Epoch 00000 | Loss 0.0955
Run 0 | Step 2226 | Epoch 00000 | Loss 0.0893
Run 0 | Step 2232 | Epoch 00000 | Loss 0.0937
Run 0 | Step 2238 | Epoch 00000 | Loss 0.1326
Run 0 | Step 2244 | Epoch 00000 | Loss 0.1339
Run 0 | Step 2250 | Epoch 00000 | Loss 0.1965
Run 0 | Step 2256 | Epoch 00000 | Loss 0.1207
Run 0 | Step 2262 | Epoch 00000 | Loss 0.1207
Run 0 | Step 2268 | Epoch 00000 | Loss 0.1091
Run 0 | Step 2274 | Epoch 00000 | Loss 0.1391
Run 0 | Step 2280 | Epoch 00000 | Loss 0.1303
Run 0 | Step 2286 | Epoch 00000 | Loss 0.1522
Run 0 | Step 2292 | Epoch 00000 | Loss 0.1046
Run 0 | Step 2298 | Epoch 00000 | 

Run 0 | Step 3246 | Epoch 00000 | Loss 0.1441
Run 0 | Step 3252 | Epoch 00000 | Loss 0.1490
Run 0 | Step 3258 | Epoch 00000 | Loss 0.1702
Run 0 | Step 3264 | Epoch 00000 | Loss 0.1091
Run 0 | Step 3270 | Epoch 00000 | Loss 0.1321
Run 0 | Step 3276 | Epoch 00000 | Loss 0.0976
Run 0 | Step 3282 | Epoch 00000 | Loss 0.1275
Run 0 | Step 3288 | Epoch 00000 | Loss 0.0619
Run 0 | Step 3294 | Epoch 00000 | Loss 0.1642
Run 0 | Step 3300 | Epoch 00000 | Loss 0.0996
Run 0 | Step 3306 | Epoch 00000 | Loss 0.0904
Run 0 | Step 3312 | Epoch 00000 | Loss 0.0936
Run 0 | Step 3318 | Epoch 00000 | Loss 0.1301
Run 0 | Step 3324 | Epoch 00000 | Loss 0.1304
Run 0 | Step 3330 | Epoch 00000 | Loss 0.1550
Run 0 | Step 3336 | Epoch 00000 | Loss 0.1114
Run 0 | Step 3342 | Epoch 00000 | Loss 0.1751
Run 0 | Step 3348 | Epoch 00000 | Loss 0.1258
Run 0 | Step 3354 | Epoch 00000 | Loss 0.1195
Run 0 | Step 3360 | Epoch 00000 | Loss 0.1000
Run 0 | Step 3366 | Epoch 00000 | Loss 0.1269
Run 0 | Step 3372 | Epoch 00000 | 

Run 0 | Step 4320 | Epoch 00000 | Loss 0.2192
Run 0 | Step 4326 | Epoch 00000 | Loss 0.1839
Run 0 | Step 4332 | Epoch 00000 | Loss 0.2395
Run 0 | Step 4338 | Epoch 00000 | Loss 0.2294
Run 0 | Step 4344 | Epoch 00000 | Loss 0.1998
Run 0 | Step 4350 | Epoch 00000 | Loss 0.2664
Run 0 | Step 4356 | Epoch 00000 | Loss 0.1826
Run 0 | Step 4362 | Epoch 00000 | Loss 0.1978
Run 0 | Step 4368 | Epoch 00000 | Loss 0.2334
Run 0 | Step 4374 | Epoch 00000 | Loss 0.2016
Run 0 | Step 4380 | Epoch 00000 | Loss 0.2023
Run 0 | Step 4386 | Epoch 00000 | Loss 0.1812
Run 0 | Step 4392 | Epoch 00000 | Loss 0.1949
Run 0 | Step 4398 | Epoch 00000 | Loss 0.2153
Run 0 | Step 4404 | Epoch 00000 | Loss 0.2551
Run 0 | Step 4410 | Epoch 00000 | Loss 0.3101
Run 0 | Step 4416 | Epoch 00000 | Loss 0.2400
Run 0 | Step 4422 | Epoch 00000 | Loss 0.2629
Run 0 | Step 4428 | Epoch 00000 | Loss 0.2082
Run 0 | Step 4434 | Epoch 00000 | Loss 0.2688
Run 0 | Step 4440 | Epoch 00000 | Loss 0.1645
Run 0 | Step 4446 | Epoch 00000 | 

Run 0 | Step 5394 | Epoch 00000 | Loss 0.1943
Run 0 | Step 5400 | Epoch 00000 | Loss 0.2055
Run 0 | Step 5406 | Epoch 00000 | Loss 0.1857
Run 0 | Step 5412 | Epoch 00000 | Loss 0.1793
Run 0 | Step 5418 | Epoch 00000 | Loss 0.1486
Run 0 | Step 5424 | Epoch 00000 | Loss 0.1650
Run 0 | Step 5430 | Epoch 00000 | Loss 0.1674
Run 0 | Step 5436 | Epoch 00000 | Loss 0.1840
Run 0 | Step 5442 | Epoch 00000 | Loss 0.2110
Run 0 | Step 5448 | Epoch 00000 | Loss 0.2095
Run 0 | Step 5454 | Epoch 00000 | Loss 0.2152
Run 0 | Step 5460 | Epoch 00000 | Loss 0.1862
Run 0 | Step 5466 | Epoch 00000 | Loss 0.1993
Run 0 | Step 5472 | Epoch 00000 | Loss 0.2918
Run 0 | Step 5478 | Epoch 00000 | Loss 0.2010
Run 0 | Step 5484 | Epoch 00000 | Loss 0.2679
Run 0 | Step 5490 | Epoch 00000 | Loss 0.2871
Run 0 | Step 5496 | Epoch 00000 | Loss 0.2372
Run 0 | Step 5502 | Epoch 00000 | Loss 0.2567
Run 0 | Step 5508 | Epoch 00000 | Loss 0.2489
Run 0 | Step 5514 | Epoch 00000 | Loss 0.3187
Run 0 | Step 5520 | Epoch 00000 | 

Run 0 | Step 6468 | Epoch 00000 | Loss 0.2120
Run 0 | Step 6474 | Epoch 00000 | Loss 0.1890
Run 0 | Step 6480 | Epoch 00000 | Loss 0.1757
Run 0 | Step 6486 | Epoch 00000 | Loss 0.2906
Run 0 | Step 6492 | Epoch 00000 | Loss 0.3250
Run 0 | Step 6498 | Epoch 00000 | Loss 0.2535
Run 0 | Step 6504 | Epoch 00000 | Loss 0.3389
Run 0 | Step 6510 | Epoch 00000 | Loss 0.3530
Run 0 | Step 6516 | Epoch 00000 | Loss 0.2484
Run 0 | Step 6522 | Epoch 00000 | Loss 0.2960
Run 0 | Step 6528 | Epoch 00000 | Loss 0.2166
Run 0 | Step 6534 | Epoch 00000 | Loss 0.3054
Run 0 | Step 6540 | Epoch 00000 | Loss 0.2467
Run 0 | Step 6546 | Epoch 00000 | Loss 0.2506
Run 0 | Step 6552 | Epoch 00000 | Loss 0.2492
Run 0 | Step 6558 | Epoch 00000 | Loss 0.2750
Run 0 | Step 6564 | Epoch 00000 | Loss 0.2343
Run 0 | Step 6570 | Epoch 00000 | Loss 0.3002
Run 0 | Step 6576 | Epoch 00000 | Loss 0.2920
Run 0 | Step 6582 | Epoch 00000 | Loss 0.2523
Run 0 | Step 6588 | Epoch 00000 | Loss 0.2589
Run 0 | Step 6594 | Epoch 00000 | 

Run 0 | Step 7542 | Epoch 00000 | Loss 0.1285
Run 0 | Step 7548 | Epoch 00000 | Loss 0.1406
Run 0 | Step 7554 | Epoch 00000 | Loss 0.1527
Run 0 | Step 7560 | Epoch 00000 | Loss 0.1147
Run 0 | Step 7566 | Epoch 00000 | Loss 0.1310
Run 0 | Step 7572 | Epoch 00000 | Loss 0.2005
Run 0 | Step 7578 | Epoch 00000 | Loss 0.2188
Run 0 | Step 7584 | Epoch 00000 | Loss 0.2039
Run 0 | Step 7590 | Epoch 00000 | Loss 0.2010
Run 0 | Step 7596 | Epoch 00000 | Loss 0.1667
Run 0 | Step 7602 | Epoch 00000 | Loss 0.2227
Run 0 | Step 7608 | Epoch 00000 | Loss 0.1770
Run 0 | Step 7614 | Epoch 00000 | Loss 0.2421
Run 0 | Step 7620 | Epoch 00000 | Loss 0.2244
Run 0 | Step 7626 | Epoch 00000 | Loss 0.1995
Run 0 | Step 7632 | Epoch 00000 | Loss 0.2079
Run 0 | Step 7638 | Epoch 00000 | Loss 0.2457
Run 0 | Step 7644 | Epoch 00000 | Loss 0.2187
Run 0 | Step 7650 | Epoch 00000 | Loss 0.1995
Run 0 | Step 7656 | Epoch 00000 | Loss 0.2214
Run 0 | Step 7662 | Epoch 00000 | Loss 0.1448
Run 0 | Step 7668 | Epoch 00000 | 

Run 0 | Step 8616 | Epoch 00000 | Loss 0.0944
Run 0 | Step 8622 | Epoch 00000 | Loss 0.1275
Run 0 | Step 8628 | Epoch 00000 | Loss 0.1388
Run 0 | Step 8634 | Epoch 00000 | Loss 0.1457
Run 1 | Step 0 | Epoch 00000 | Loss 0.0114
Run 1 | Step 6 | Epoch 00000 | Loss 0.0139
Run 1 | Step 12 | Epoch 00000 | Loss 0.0401
Run 1 | Step 18 | Epoch 00000 | Loss 0.0350
Run 1 | Step 24 | Epoch 00000 | Loss 0.0552
Run 1 | Step 30 | Epoch 00000 | Loss 0.0935
Run 1 | Step 36 | Epoch 00000 | Loss 0.0836
Run 1 | Step 42 | Epoch 00000 | Loss 0.1003
Run 1 | Step 48 | Epoch 00000 | Loss 0.1599
Run 1 | Step 54 | Epoch 00000 | Loss 0.1610
Run 1 | Step 60 | Epoch 00000 | Loss 0.1461
Run 1 | Step 66 | Epoch 00000 | Loss 0.1437
Run 1 | Step 72 | Epoch 00000 | Loss 0.1490
Run 1 | Step 78 | Epoch 00000 | Loss 0.1719
Run 1 | Step 84 | Epoch 00000 | Loss 0.2339
Run 1 | Step 90 | Epoch 00000 | Loss 0.2066
Run 1 | Step 96 | Epoch 00000 | Loss 0.2134
Run 1 | Step 102 | Epoch 00000 | Loss 0.2197
Run 1 | Step 108 | Epoch 

In [ ]:
for param in net.named_parameters():
    if param[0] == 'gcn.predict_leaving.embed.weight':
        break


In [ ]:
E = param[1][:]

In [ ]:
E[123]

In [ ]:
with open('embeddings_GATFlow_4step_10s_mse_002.pkl','wb') as f:
    pickle.dump(E, f)